We have an ASCII file defining every building ground plan as a polygon by listing its vertices. Hence the entry for a given building is the building number followed by the number of vertices of its boundary and the rooftop elevation (assumed flat), followed by a list of the (X,Y) coordinates of each one of the vertices of its boundary. As an example the entry for building number 186 is given below:
The sequence means: Building number 186 is a polygon with 4 vertices and its rooftop elevation is 67.5 m (MSL). The (X,Y) co-ordinates of the given 4 vertices follow in the four next lines.

```
186    4      67.5

       2100.75      3819.50   

       2105.56      3820.50   

       2100.50      3816.00   

       2097.00      3819.50   
....
```

In [43]:
input = 'Buildings.dat'
output = 'Buildings.csv'

In [41]:
data = []
with open(input, 'r') as f:
    for line in f:
        # Get number of verticies from the first line
        fid, numvertices, elev = line.split()
        coordinates = []
        # Skip ahead number of lines equal to number of vertices and save the coordinates
        for x in range(int(numvertices)):
            x, y = f.readline().split()
            coordinates.append(('{} {}'.format(x,y)))
        # Discard first coordinate which is the centroid
        wkt = 'POLYGON (({}))'.format(','.join(coordinates[1:]))
        data.append({'fid': int(fid), 'elev': float(elev), 'wkt': wkt})

In [42]:
import csv

with open(output, 'w') as csvfile:
    fieldnames = ['fid', 'elev', 'wkt']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in data:
        writer.writerow(row)
